Marcin Wardyński  
czwartek, 8:00

Zgodnie z poleceniem przetestuję następujące zdania z maskowaniem:

In [34]:
examples = {"PRZYPADKI": [
        "Ta >>><<< jest bardzo mocno napompowana.",
        "Naucz się >>><<< na jutrzejszy sprawdzian.",
        "Prawdziwemu >>><<< zawsze można ufać.",
        "W deszczowy dzień zabierz ze sobą >>><<<.",
        "Uwielbiam spędzać czas z >>><<<.",
        "Opowiadanie o >>><<< zawsze doprowadza mnie do łez.",
        "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..."
    ],
    "RELACJA": [
        "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze.",
        "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie.",
        "Gdy już go znajdziesz, zrób z >>><<< co uważasz za stosowne."
    ],
    "WIEDZA": [
        "Stolicą Niemiec jest >>><<<.",
        "Ziemia jest >>><<< planetą od Słońca.",
        ">>><<< jest procesem zmiany ze stanu ciekłego w stan gazowy.",
        "Wisła uchodzi do >>><<<."
    ],
    "NACECHOWANIE": [
        "'Jestem w niej zakochany na zabój'  jest zdaniem wyrażającym >>><<<.",
        "'Motyla noga, nigdy nie znajdę już tego świerszcza' powiedziała osoba o nastawieniu >>><<<.",
        "'Jak ja nie cierpię tych okropnych Smerfów' to zdanie przedstawiające >>><<< emocje.",
        "'Hura, nareszcie zdałem do kolejnej klasy' powiedziała osoba czująca się >>><<<.",
        "'Niech on tylko wpadnie w moje ręce' spośród emocji pozytywna, neutralna i negatywna, najbliżej temu zdaniu jest emocja >>><<<."
    ]}


Do sprawnego odpytywania modeli językowych o ich odpowiedz posłuży mi poniższa funkcja:

In [35]:
from transformers import pipeline

def evaluate_lm(cat_sentences, model):
    mask_holder = ">>><<<"
    unmasker = pipeline('fill-mask', model=model, device=0)
    
    for cat, sentences in cat_sentences.items():
        print()
        for sentence in sentences:
            masked_sentence = sentence.replace(mask_holder, unmasker.tokenizer.mask_token)
            unmasked_tokens = unmasker(masked_sentence)
            tokens = [token["token_str"] for token in unmasked_tokens]
            print(f"{cat}: \"{sentence}\" <- {tokens}")

W teście wezmą cztery modele językowe z huggingface.com:
- https://huggingface.co/dkleczek/bert-base-polish-cased-v1
- https://huggingface.co/jhu-clsp/bernice
- https://huggingface.co/deepsense-ai/trelbert
- https://huggingface.co/FacebookAI/xlm-roberta-base


W ramach ćwiczenia sprawdzilem również następujące modele:

- https://huggingface.co/google-bert/bert-base-multilingual-uncased
- https://huggingface.co/Twitter/twhin-bert-base
- https://huggingface.co/FacebookAI/xlm-mlm-17-1280
- https://huggingface.co/studio-ousia/mluke-base
- https://huggingface.co/sdadas/polish-longformer-base-4096
- https://huggingface.co/sdadas/polish-splade
- https://huggingface.co/facebook/xlm-v-base
- https://huggingface.co/joelniklaus/legal-xlm-longformer-base
- https://huggingface.co/EuropeanParliament/EUBERT
- https://huggingface.co/OrlikB/KartonBERT_base_uncased_v1

Ich odpowiedzi są załączone na końcu notatnika, nie zostały uwzględnione przy interpretacji wyników.

#### Wyniki

Ponieżej znajdują się wyniki dla utworzonych zapytań każdego z modeli, ale zanim przejdziemy do omawiania wyników każdego z osobna, przedstawie w prostej skali punktowej ich efektywność w każdej z kategorii. Punkty przyznaję w następujący sposób:
- za każdy przypadek 1p. gdy co najmniej trzy odpowiedzi pasują do zadanego przypaku,
- za każdą relację long-range 1p, jeśli wśród odpowiedzi znajduje się ta poprawna,
- za każdą poprawną odpowiedź na pytanie z wiedzy ogólnej 1p. Wystarczy że odpowiedź ta będzie wśród TOP5.
- za każde poprawnie ocenione nacechowanie 1p. Znów wystarczy, że zdatna odpowiedź znajduje się w TOP5.

|            | bert-base-polish-cased-v1 | trelbert | xlm-roberta-base  | bernice |
|------------|---------------------------|----------|-------------------|---------|
| Przypadki  | 7/7                       | 7/7      | 6/7               | 5/7     |
| Long-range | 3/3                       | 3/3      | 3/3               | 2/3     |
| Wiedza     | 2/4                       | 1/4      | 1/4               | 0/4     |
| Sentyment  | 4/5                       | 4/5      | 2/5               | 1/5     |

Patrząc na sumaryczne wyniki najlepiej wypadł `bert-base`, wyprzedzając jedynie o 1p. `trelbert`. Model `xlm-roberta` ustępuje pierwszej dwójce a `bernice` pozostaje daleko w tyle, chociaż jest lepsza od więkoszości odrzuconych modeli.

Wszystkie modele poradziły sobie z przypadkami bardzo dobrze. `bert-base` i `trelbert` były bezbłędne, `xlm-roberta` nie poradziła sobie wystarczająco z wołaczem, a `bernice` poza brakami w wołaczu, poległa również na celowniku.

Jeśli chodzi o zachowanie relacji long-range dla podmiotu, to właściwie wszystkie modele poradziły sobie bardzo dobrze, jedynie `bernice` nie popełniła błąd, reszta była bezbłędna.

W przypadku wiedzy o świecie, to tutaj nasze podstawowe modele nie wypadły najlepiej. `bert-base` dał dobrą odpowiedź na połowę pytań, `trelbert` i `xlm-roberta` tylko na jedno, a `bernice` nie dała żadnej prawidłowej odpowiedzi.

Klasyfikacja zero-shot dla sentymentu przebiegła dla `bert-base` i `trelbert` bardzo dobrze, gdyż właściwa odpowiedź nie została uwzględniona w zbiorze odpowiedzi tylko raz. `xlm-roberta` zawarła w zbiorze odpowiedzi dwie poprawne odpowiedzi, a `bernice` tylko jedną. Dodam tylko, że przy ocenie klasyfikacji sentymentu byłem dość pobłażliwy dla wszystkich modeli.

Więcej na temat każdej z ocenianych kategorii, jak i nabardziej jaskrawych błędów można przeczytać w sekcji dotyczącej każdego modelu z osobna.

#### bert-base-polish-cased-v1

In [36]:
#https://huggingface.co/dkleczek/bert-base-polish-cased-v1
evaluate_lm(examples, 'dkleczek/bert-base-polish-cased-v1')


PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['woda', 'rzeka', 'ziemia', 'energia', 'linia']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['tego', 'chodzić', 'angielskiego', 'liczyć', 'czegoś']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['człowiekowi', 'nie', 'Bogu', 'królowi', 'przyjacielowi']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['dzieci', 'psa', 'konie', 'ludzi', 'dziecko']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['dziećmi', 'tobą', 'przyjaciółmi', 'rodziną', 'ludźmi']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['tym', 'nim', 'niej', 'sobie', 'tobie']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['Komisjo', 'pani', 'Izbo', 'Pani', 'Komisja']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['to', 'ją', 'co', 'mnie', 'coś']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['im', 'temu', 'jej', 'mu', 'ich']
R

Przypadki wręcz wzorowo, jedynie w bierniku zbudowane zdanie nie miałoby dla nas sensu, ale gramatycznie jest poprawne.

Przy relacjach long-range, dla pierwszego zdania optymalna odpowiedź znajduje się na drugim miejscu, dla pozostałych pierwsza odpowiedź jest zarazem tą najlepszą.

Wiedza nie jest mocną stroną tego modelu, chociaż i tak wypadł najlepiej z badanych. Dwa pytania nie uzyskały poprawnej odpowiedzi, a dla dwóch pozostałych poprawna odpowiedź była dopiero na trzecim miejscu wśród propozycji. Mimo tego, poprawna odpowiedź o uchodzenie Wisły do Bałtyku zrobiła na mnie bardzo dobre wrażenie.

Co do klasyfikacji sentymentu, to nie upierałem się przy zdefiniowanych uprzednio klasach, a chciałem zobaczyć wolne podejście modelu. W pierwszym zdaniu `nienawiść` zdecydowanie nie pasuje do reszty, drugie okazało się zbyt trudne do sklasyfikowania, trzecie zawiera odpowiedź klasyfikującą zdanie negatywnie na czwartej pozycji, lecz odpowiedź numer 5 uważa odwrotnie. Zdanie czwartek ma dobre odpowiedzi na pozycjach 1 i 3, natomiast pozostałe tu nie pasują, a zdanie piąte otrzymało poprawną odpowiedź, czyli `negatywna` na pozycji 1, ale `pozytywna` była zaraz za nią. Przynajmniej model skorzystał z sugerowanych klas sentymentu, z czym pozostałe modele miały problem.

Ewidentnie nie widać spójności w klasyfikacji sentymentu. Skrajnie przeciwne klasy wyceniane są w zbiorze odpowiedzi jedna za drugą. Jest to największa bolączka tego modelu na równi z jednak dość ograniczoną wiedzą o świecie.

#### trelbert

In [51]:
#https://huggingface.co/deepsense-ai/trelbert
evaluate_lm(examples, 'deepsense-ai/trelbert')


PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['pani', 'kobieta', 'piłka', 'strona', 'Pani']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['polskiego', 'angielskiego', 'hiszpańskiego', 'pisać', 'niemieckiego']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['człowiekowi', 'Bogu', 'Panu', 'prezydentowi', 'panu']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['rower', 'psa', 'plecak', 'czapkę', 'kota']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['rodziną', 'ludźmi', 'dziećmi', 'przyjaciółmi', 'psem']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['tym', 'nim', 'nich', 'sobie', 'tobie']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['Pani', 'pani', '@anonymized_account', 'Państwo', 'Rada']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['ją', 'się', 'to', 'czy', 'go']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['im', 'sobie',

Przypadki jak w przypadku `bert-base` świetnie, jedynie zdania w bierniku nie mają sensu. Poza tym nie wszystkie odpowiedzi dla wołacza są odpowiednie, ale za to propozycje dla dopełniacza wyglądają najlepiej wśród wszystkich badanych modeli.

Relacje long-range bezbłędnie z poprawnymi odpowiedziami na pierwszym miejscu.

Z wiedzy 5 za geografię i podanie Berlina jako pierwszej odpowiedzi na pytanie o stolicę Niemiec. Następne pytania dość kiepsko, a przy ostatnim o Wisłę, model zapewnie pomylił rzekę z klubem piłkarskim.

Co do badania sentymentu, to widać pewne zbierzności z `base-bert`. W pierwszym również zdecydowanie nie pasuje `nienawiść`, najpewnie modele dały się złapać przez dość przekorne sformułowanie "na zabój". Drugie jest kompletnie źle. Trzecie z odpowiedzią z czwartego miejsca `najgorsze`, zostało poprawnie sklasyfikowane negatywnie, pozostałe odpowiedzi są akceptowalne, ale słabo klasyfikują sentyment. Dla zdania czwartego, odpowiedzi od trzeciej są poprawne, pierwsza i druga są poniekąd ich przeciwieństwem i nie pasują po wypowiedzianego zdania. Zdanie piąte dostało poprawną odpowidź `nienawiści` na miejscu pierwszym. Mankamentem jest nie skorzystanie z podpowiedzianych klas oraz usytuowaniem odpowiedzi `pozytywna` oraz `miłości` zaraz za `nienawiści`. Czyżby i dla modeli językowych granica między miłością a nienawiścią również była tak cieńka?!? XD

Generalnie, model ma podobne przypadłości co `base-bert`, przeplata opinie pozytywne z negatywnymi i mało wie o świecie.

#### xlm-roberta-base

In [52]:
#https://huggingface.co/FacebookAI/xlm-roberta-base
evaluate_lm(examples, 'xlm-roberta-base')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['część', 'cena', 'strona', 'farba', 'maska']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['przygotować', 'wszystkiego', 'odpowiedzi', 'angielskiego', 'już']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['Bogu', 'nie', 'życiu', 'dziecku', ',']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['wodę', 'śnieg', 'rower', 'kwiaty', 'słońce']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['dziećmi', 'dzieckiem', 'ludźmi', 'innymi', 'nimi']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['miłości', 'sobie', 'życiu', 'przeszłości', 'niej']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['Pani', 'Państwo', 'pani', 'Państwa', 'strona']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['ją', 'to', 'czy', 'je', 'sytuację']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['im', 'nim', 'tym', 'sobie', 

Przypadki bardzo dobrze, jak zwykle sugestie w bierniku nie mają sensu, aczkolwiek są gramatycznie poprawne. W wołaczu tylko dwie sugestie są poprawne, stąd brak punktu.

Relacja long-range poprawnie już przy pierwszych odpowiedziach.

Model odpowiedział poprawnie tylko na pytanie o stolicę Niemiec, ale nie była ona na miejscu pierwszym. Pozostałe odpowiedzi są błędne.

Dla pierwszego zdania wyrażającego emocje w zasadzie wszystkie zaproponowane odpowiedzi są poprawne. Drugie nie ma poprawnych sugestii, a co gorsze, model sugeruje drugą kropkę na końcu zdania. Dla trzeciego zdania brak klasyfikacji emocji. Czwarte ma wszytkie odpowiedzi poprawne. Piąte miesza języki, faktycznie proponuje poprawną odpowiedź na pierwszym miejscu, lecz nie po polsku, a zaraz za nią, stawia dwukrotnie odpoiwdź przeciwną, gdzie również miesza języki. Widać staranie użycia zaproponowanych klas.

Model ten wie o świecie jeszcze mniej, ale za to rzadziej przypisuje skrajnie przeciwne klasy sentymentu tym samym zdaniom.

In [37]:
#https://huggingface.co/jhu-clsp/bernice
evaluate_lm(examples, 'jhu-clsp/bernice')

Some weights of the model checkpoint at jhu-clsp/bernice were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['kobieta', 'pani', 'Pani', 'osoba', 'lista']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['matmy', 'pisać', 'angielskiego', 'iść', 'napisać']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['nie', ',', 'Bogu', 'ludziom', 'człowieka']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['mnie', 'siebie', 'wszystko', 'jedzenie', 'życie']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['nią', 'nim', 'nimi', 'ludźmi', 'kim']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['sobie', 'miłości', 'Tobie', 'tym', 'wszystkim']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['Pani', 'pani', 'odpowiedź', 'osoba', 'opozycja']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['czy', 'się', 'to', 'cie', 'je']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['im', 'na', 'tam', 'ich', ',']
RELACJA: "Gdy już go z

Model poradził sobie z przypadkami najgorzej, lecz wciąż przyzwoicie. W celowniku tylko jedna odpowiedź jest poprawna, a w wołaczu dwie, a więc brakuje trochę do przyznania punktu. Odpowiedzi dla biernika, jak i we wszystkich powyższych modelach, są poprawne lecz nie stanowią dobrej sugestii dla czlowieka.

Relacja long-range zachowana dla dwóch z trzech przypadków, ale tylko dla drugiego zdania optymalna odpowiedź była pierwszą sugestią.

Wiedza modelu to totalna kompromitacja. 2.0, do poprawki!

Sentyment został również poprawnie sklasyfikowany w jednym przypadku, a mianowicie w zdaniu czwartym, aczkolwiek i tak jest to naciągany punkt, gdyż wśród odpowiedzi, nacechowanie pozytywne do negatywnego wystąpuje w stosunku 2:2. Pozostałe klasyfikacji nie mają kompletnie sensu, model wstawia wielokrotne znaki interpunkcji i nie uwzględnia podpowiedzi.



### Dodatkowe uwagi

- Zakończenie zdania kropką zniechęca model od wypełnienia maski znakiem interpunkcyjnym.
- Podanie możliwych odpowiedzi dla klasyfikacji sentymentu zostało wykorzystane w dwóch z czterech przypadków.

### Modele językowe wypróbowane, lecz nie zestawione w opracowaniu wyników

In [40]:
#https://huggingface.co/google-bert/bert-base-multilingual-uncased
evaluate_lm(examples, 'bert-base-multilingual-uncased')

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['wersja', 'piosenka', 'droga', 'płyta', 'ostatnia']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['tez', 'takze', 'to', 'rowniez', 'go']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['nie', ',', 'prawie', 'niemal', '-']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['dzieci', 'ludzi', 'ok', 'kobiety', 'wiecej']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['nim', 'nia', 'matka', 'rodzina', 'muzyka']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['tym', 'nim', 'sobie', 'zyciu', 'niej']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['"', '.', ')', '##ch', '##mi']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['##y', '##ono', '##ons', '##my', '##one']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['mu', 'im', 'na', 'sobie', 'juz']
RELACJA: "Gdy już go znajdziesz, zrób z >>><<< co u

In [41]:
#https://huggingface.co/Twitter/twhin-bert-base
evaluate_lm(examples, 'Twitter/twhin-bert-base')

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.



PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['sytuacja', 'szkoła', 'Polska', 'Ukraina', 'książka']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['już', 'jutro', 'dzisiaj', 'dziś', 'teraz']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['nie', 'się', ',', 'to', 'na']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['wodę', 'Polskę', 'kwiaty', 'miłość', 'siebie']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['ludźmi', 'innymi', 'nimi', 'nim', 'sobą']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['mnie', 'pracę', 'sobie', 'pracy', 'tym']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['praca', 'jestem', 'miłość', 'rzecz', 'Polska']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['się', 'ją', 'je', 'to', 'sie']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['aż', 'nawet', 'im', 'jakoś', ',']
RELACJA: "Gdy już go znajdziesz, zrób z

In [42]:
#https://huggingface.co/FacebookAI/xlm-mlm-17-1280
evaluate_lm(examples, 'xlm-mlm-17-1280')



PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['♠', '₁', '♣', '̄', 'rima']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['kończy', 'zmienia', 'gra', '↘', 'dało']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['↘', '⑩', '⑤', '█', '④']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['rzadko', 'itd', 'wiosną', 'będzie', '2-2']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['powrotem', 'wodą', 'przyszłości', 'kością', 'każdym']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['mnie', 'czym', '♣', 'miłości', 'drugą']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['↘', '►', '➜', '◄', '↓']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['esz', 'się', 'taką', 'gum', 'głowę']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['niczym', 'dając', 'poprzez', 'żeby', 'odpowiada']
RELACJA: "Gdy już go znajdziesz, zrób z >>><<< co uważasz za stosown

In [43]:
#https://huggingface.co/studio-ousia/mluke-base
evaluate_lm(examples, 'studio-ousia/mluke-base')

Some weights of the model checkpoint at studio-ousia/mluke-base were not used when initializing LukeForMaskedLM: ['luke.encoder.layer.0.attention.self.e2e_query.bias', 'luke.encoder.layer.0.attention.self.e2e_query.weight', 'luke.encoder.layer.0.attention.self.e2w_query.bias', 'luke.encoder.layer.0.attention.self.e2w_query.weight', 'luke.encoder.layer.0.attention.self.w2e_query.bias', 'luke.encoder.layer.0.attention.self.w2e_query.weight', 'luke.encoder.layer.1.attention.self.e2e_query.bias', 'luke.encoder.layer.1.attention.self.e2e_query.weight', 'luke.encoder.layer.1.attention.self.e2w_query.bias', 'luke.encoder.layer.1.attention.self.e2w_query.weight', 'luke.encoder.layer.1.attention.self.w2e_query.bias', 'luke.encoder.layer.1.attention.self.w2e_query.weight', 'luke.encoder.layer.10.attention.self.e2e_query.bias', 'luke.encoder.layer.10.attention.self.e2e_query.weight', 'luke.encoder.layer.10.attention.self.e2w_query.bias', 'luke.encoder.layer.10.attention.self.e2w_query.weight', 'l


PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['pompa', 'bomba', 'część', 'droga', 'lampa']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['go', 'czasu', 'tego', 'mnie', 'gry']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['nie', 'życiu', 'Bogu', 'prawie', 'niemal']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['deszcz', 'wiatr', 'śnieg', ':', 'rok']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['dziećmi', 'ludźmi', 'dzieckiem', 'wodą', 'sobą']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['miłości', 'tym', 'mnie', 'nim', 'niej']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['strona', 'odpowiedź', 'decyzja', 'jest', ',']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['ją', 'się', 'to', 'mnie', 'jej']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['im', 'tym', 'ich', 'nim', 'temu']
RELACJA: "Gdy już go znajdziesz, zrób z >>

In [44]:
#https://huggingface.co/sdadas/polish-longformer-base-4096
evaluate_lm(examples, 'sdadas/polish-longformer-base-4096')

Some weights of the model checkpoint at sdadas/polish-longformer-base-4096 were not used when initializing LongformerForMaskedLM: ['longformer.embeddings.position_ids']
- This IS expected if you are initializing LongformerForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['woda', 'ziemia', 'rzeka', 'sala', 'butelka']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['pisać', 'chodzić', 'odpowiadać', 'przygotowywać', 'angielskiego']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['człowiekowi', 'Bogu', 'nie', 'mężowi', 'królowi']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['parasol', 'wodę', 'płaszcz', 'deszcz', 'kapelusz']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['tobą', 'dziećmi', 'nimi', 'ludźmi', 'Tobą']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['tym', 'nim', 'nich', 'sobie', 'miłości']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['Pani', 'pani', 'Premier', 'Rado', 'Minister']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['coś', 'raz', 'to', 'ją', 'j']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['im', 'jej', 'nikomu', 'temu', 'mu'

In [45]:
#https://huggingface.co/sdadas/polish-splade
evaluate_lm(examples, 'sdadas/polish-splade')


PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['łódź', 'Titan', '164', 'rzeka', 'cover']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['cierpliwość', 'taktyki', 'poprawnie', 'metody', 'pisanie']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['uczciwość', 'zaufanie', 'prawdy', 'prawdziwe', '183']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['ubrania', 'plecak', 'chleb', 'prezenty', 'rower']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['dziewczyny', 'kobiety', 'Paris', 'Sophie', 'ciebie']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['gwałt', 'Spider', 'autor', 'Potter', 'history']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['uprzejmie', 'pani', '183', 'prośby', 'Sophie']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['paczki', 'zawartość', 'odbiór', 'etykiet', 'sprawdzenie']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <-

In [46]:
#https://huggingface.co/facebook/xlm-v-base
evaluate_lm(examples, 'facebook/xlm-v-base')


PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['woda', 'kawa', 'zupa', 'skóra', 'ryba']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['liczyć', 'przygotować', 'pisać', 'odpowiadać', 'matematyki']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['Bogu', 'nie', 'człowiekowi', 'dziecku', 'Panu']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['słońce', 'książkę', 'śnieg', 'wodę', 'kwiaty']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['przyjaciółmi', 'rodziną', 'dziećmi', 'dzieckiem', 'ludźmi']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['miłości', 'tobie', 'sobie', 'przeszłości', 'ludziach']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['Pani', 'pani', 'Państwo', 'Panie', 'Panowie']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['ją', 'czy', 'się', 'je', 'to']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['im', 'nim', 'tym', 

In [47]:
#https://huggingface.co/OrlikB/KartonBERT_base_uncased_v1
evaluate_lm(examples, 'OrlikB/KartonBERT_base_uncased_v1')


PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['strona', 'maszyna', 'dziewczyna', 'scena', 'kobieta']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['chodzić', 'jeździć', 'materiału', 'iść', 'języka']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['człowiekowi', 'bogu', 'panu', 'światu', 'życiu']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['broń', 'plecak', 'rower', 'jedzenie', 'samochód']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['rodziną', 'przyjaciółmi', 'tobą', 'dziećmi', 'ludźmi']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['tym', 'mnie', 'nim', 'sobie', 'tobie']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['pani', 'państwo', 'panie', 'spółka', 'redakcja']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['ją', 'to', 'go', 'coś', 'je']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['im', 'temu', 'ludziom', 'tym',

In [48]:
#https://huggingface.co/joelniklaus/legal-xlm-longformer-base
evaluate_lm(examples, 'joelniklaus/legal-xlm-longformer-base')


PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- [' nie', ' sytuacja', ' sprawa', ' społka', 'cja']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- [' wiecej', ' tez', ' takze', ' rowniez', ' tylko']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- [' sobie', ' panstwu', ' nie', ' obrazu', ',']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- [' ”', ' "', '!', '?', ' »']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- [' niemiec', ' polska', ' belgii', ' luksemburga', ' polski']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- [' tym', ' nim', ' nie', ' polsce', ' kosztach']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- [' pani', ' pan', ' minister', ' ministra', ' pana']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['anie', ' ja', ' sie', ' je', 'cie']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- [' juz', ' tu', ' na', ' do', ' mi']
RELACJA

In [49]:
#https://huggingface.co/EuropeanParliament/EUBERT
evaluate_lm(examples, 'EuropeanParliament/EUBERT')


PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['ta', "'", 'nie', '’', '”']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['##c', '##ca', '##wa', '##pic', '##cza']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['.', 'nie', 'mozna', 'sa', ',']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['m', 'h', 'd', 'a', 's']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['1', '2', 'r', '3', '8']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['wartosci', 'tej', 'charakterze', 'niej', 'tym']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['republika', 'panstwa', 'strone', 'pytanie', 'region']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['##ajac', '##ac', '##ie', '##a', '##i']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['sie', '.', 'na', ',', 'rowniez']
RELACJA: "Gdy już go znajdziesz, zrób z >>><<< co uważasz za stosowne." <- ['tego', 'ty

In [50]:
#https://huggingface.co/OrlikB/KartonBERT_base_uncased_v1
evaluate_lm(examples, 'OrlikB/KartonBERT_base_uncased_v1')


PRZYPADKI: "Ta >>><<< jest bardzo mocno napompowana." <- ['strona', 'maszyna', 'dziewczyna', 'scena', 'kobieta']
PRZYPADKI: "Naucz się >>><<< na jutrzejszy sprawdzian." <- ['chodzić', 'jeździć', 'materiału', 'iść', 'języka']
PRZYPADKI: "Prawdziwemu >>><<< zawsze można ufać." <- ['człowiekowi', 'bogu', 'panu', 'światu', 'życiu']
PRZYPADKI: "W deszczowy dzień zabierz ze sobą >>><<<." <- ['broń', 'plecak', 'rower', 'jedzenie', 'samochód']
PRZYPADKI: "Uwielbiam spędzać czas z >>><<<." <- ['rodziną', 'przyjaciółmi', 'tobą', 'dziećmi', 'ludźmi']
PRZYPADKI: "Opowiadanie o >>><<< zawsze doprowadza mnie do łez." <- ['tym', 'mnie', 'nim', 'sobie', 'tobie']
PRZYPADKI: "Szanowna >>><<<, nawiązując do otrzymanych zastrzeżeń..." <- ['pani', 'państwo', 'panie', 'spółka', 'redakcja']

RELACJA: "Gdy odbierasz paczkę, sprawdź >>><<< jeszcze przy kurierze." <- ['ją', 'to', 'go', 'coś', 'je']
RELACJA: "Mijając ludzi na ulicy nie przyglądaj się >>><<< zbyt natarczywie." <- ['im', 'temu', 'ludziom', 'tym',